In [2]:
#!pip install us

In [3]:
from bs4 import BeautifulSoup
import requests
import ibm_db
import us
import os
import pandas as pd
import numpy as np

<h3>Database Connection</h3>
Connects to IBM cloud DB2 database

In [73]:
dsn_hostname = "98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud"
dsn_uid = "xcb12089"
dsn_pwd = "xev1HxMBa4AgFJFV"
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"
dsn_port = "30875"
dsn_protocol = "TCPIP"
dsn_security = "SSL"

dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, 
                            dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  xcb12089 on host:  98538591-7217-4024-b027-8baa776ffad1.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud


<h3>Webscraping</h3>
Retrieves the financial info for the largest companies by revenue

In [4]:
URL = "https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue"
html_data = requests.get(URL).text

In [5]:
soup = BeautifulSoup(html_data,'html.parser')
data = pd.DataFrame(columns=["Name","Symbol","Industry","Revenue(USD Millions)",
                             "Revenue_Growth","Operating_Income","Net_Income",
                             "Assets","Equity","Employees","Headquarters"])
profit_data = pd.DataFrame(columns=["Name","Profit"])

In [6]:
for row in soup.find_all('tbody')[1].find_all('tr'):
    col = row.find_all('td')
    i = -1
    symbol = 'Null'
    o_income = 'Null'
    n_income = 'Null'
    assets = 'Null'
    equity = 'Null'
    if (col != []):
        name = col[1].text
        industry = col[2].text
        revenue = col[3].text
        pct = col[4].text
        employees = col[5].text
        headquarters = col[6].text
        sublink = "https://en.wikipedia.org{}".format(row.find_all('a')[0].get('href')) 
        sub_html = requests.get(sublink).text
        soup_sublink = BeautifulSoup(sub_html,'html.parser')
        infobox = soup_sublink.find("table", {"class": "infobox vcard"})
        for row in infobox.find_all('tr'): #Gets stock symbol and financial information from the sublinks
            col = row.find_all('th')
            i = i + 1
            if (col != [] and col[0].text == 'Traded as'):
                symbol = infobox.find_all('tr')[i].find_all('a')[2].text
            if (col != [] and col[0].text == 'Operating income'):
                o_income = infobox.find_all('tr')[i].find_all('td')[0].text
            if (col != [] and col[0].text == 'Net income'):
                n_income = infobox.find_all('tr')[i].find_all('td')[0].text
            if (col != [] and col[0].text == 'Total assets'):
                assets = infobox.find_all('tr')[i].find_all('td')[0].text
            if (col != [] and col[0].text == 'Total equity'):
                equity = infobox.find_all('tr')[i].find_all('td')[0].text
        data=data.append({'Name':name,'Symbol':symbol,'Industry':industry,'Revenue(USD Millions)':revenue,
                          'Revenue_Growth':pct,'Operating_Income':o_income,'Net_Income':n_income,
                          'Assets':assets,'Equity':equity,'Employees':employees,
                          'Headquarters':headquarters}, ignore_index = True)

In [7]:
#Retrieves data from the largest companies by profit table
for row in soup.find_all('tbody')[2].find_all('tr'):
    col = row.find_all('td')
    if (col != []):
        name = col[1].text
        profit = col[3].text
        profit_data=profit_data.append({'Name':name,'Profit':profit}, ignore_index=True)

<h3>Data Cleaning</h3>
Cleans and modifies the data to prepare it for DB2 upload

In [8]:
#Removes the newline symbols
data = data.replace('\n','',regex=True)
profit_data = profit_data.replace('\n','',regex=True)

In [ ]:
#Initial data to CSV
path=os.path.join(os.getcwd(),'Initial_Data.csv')
path
csv = data.to_csv(path)

In [11]:
financials_type = ['Operating_Income','Net_Income','Assets','Equity']
#removes unicode spaces to make regex work correctly
for index in range(len(financials_type)):
    t=financials_type[index]
    data[t] = data[t].str.replace(u'\xa0',' ') #index 75

In [12]:
count = data.Headquarters.str.count(',') #checks to make sure Headquarters can be split into two columns
data[['City','State']] = data['Headquarters'].str.split(',', expand=True,n=1)
data.iloc[count[count>1].index, 12] = data.iloc[count[count>1].index, 12].str.split(',').str[1]#used to get state when there is a county

In [13]:
#Transformation from percentage to decimal
data['Revenue_Growth'] = round((data['Revenue_Growth'].str.strip('%').astype('float') / 100.0),2)

In [14]:
#Removes commas from revenue so it can be uploaded as a numeric value
data['Revenue(USD Millions)'] = data['Revenue(USD Millions)'].str.replace(',','')

In [15]:
#Separates the industries into multiple columns
industry = lambda x: 'Industry_{}'.format(x+1)
data = data.join(data['Industry'].str.split('and',expand=True).rename(columns=industry).fillna(''))

In [16]:
#Splits the information in each financial type column using regex.
#ex. US$212.206 billion[1] (2021) -> 212.206, billion, 2021
for index in range(len(financials_type)):
    t=financials_type[index]
    data[['{}_Amount'.format(t),'{}_Scale'.format(t),'{}_FY'.format(t)]] = \
    data[t].str.extract(r'\$(.+?\d.+?\s)([BMbm]illion).+?(\/*\d\d\d\d)\)',expand=True) #extracts relevant info into mutiple columns
    #Data cleansing
    data['{}_Amount'.format(t)] = data['{}_Amount'.format(t)].fillna(0)
    data['{}_Amount'.format(t)] = data['{}_Amount'.format(t)].str.strip(' ').str.strip('$')
    data['{}_Amount'.format(t)] = data['{}_Amount'.format(t)].str.replace(',','').str.replace('−','-') #Change to proper negative value unicode to ascii
    data['{}_Amount'.format(t)] = pd.to_numeric(data['{}_Amount'.format(t)])
    data['{}_Amount'.format(t)] = np.where(data['{}_Scale'.format(t)].str \
        .contains('[bB]illion',regex=True),data['{}_Amount'.format(t)].astype('float') * 1000, \
        data['{}_Amount'.format(t)].astype('float')) #Rewrites the amount in millions to match with revenue and profit

In [17]:
#Drops unused columns
data = data.drop(['Industry','Operating_Income','Net_Income','Assets','Equity','Headquarters' \
                 ,'Operating_Income_Scale','Net_Income_Scale','Assets_Scale','Equity_Scale'], axis = 1)

In [18]:
#Joins profit data to dataframe
data = data.merge(profit_data, on='Name', how='left')
data['Profit'] = data['Profit'].str.replace(',','')

In [19]:
#uses the us package to change states to their abbreviation
data['State'] = data['State'].str.replace('.','',regex=False).str.strip()
for index in range(len(data)):
    data['State'][index] = us.states.lookup(data['State'][index]).abbr

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [21]:
#Cleaned data to CSV
path=os.path.join(os.getcwd(),'Financial_Data.csv')
path
csv = data.to_csv(path)

<h3>Data Transformation</h3>
Modifies the data structure for mapping to DB2 tables

In [85]:
financials = data[['Symbol','Revenue(USD Millions)','Operating_Income_Amount',
    'Operating_Income_FY','Net_Income_Amount','Net_Income_FY','Assets_Amount',
    'Assets_FY','Equity_Amount','Equity_FY','Profit']]
Companies = data[['Symbol','Name','Industry_1','Industry_2','City','State']]

In [86]:
years = pd.DataFrame(columns=['Year'])
for i in range(len(financials_type)):
    data['{}_FY'.format(financials_type[i])] = data['{}_FY'.format(financials_type[i])].replace('[^\d]','',regex=True) #some values had /2022
    #Finds the minimum year of finanical data
    years = years.append({'Year':data['{}_FY'.format(financials_type[i])].value_counts().index.min()}, 
                 ignore_index = True)
years.min() #Account for data between 2016 and 2022
#creates dataframe for combined fiscal year data using one FY column
FY_Data = pd.DataFrame(columns=['Symbol','Fiscal Year','Operating_Income','Net_Income','Assets','Equity','Profit','Revenue'])

In [87]:
#Drop companies with no stock symbol
Companies = Companies.drop(index=(Companies[Companies['Symbol'] == 'Null']).index).reset_index()
pd.set_option("max_rows",None)
pd.set_option('max_columns',None)

In [88]:
#Adds stock symbols and years between 2016(the minium year) to 2023 (current) as well as 0000 for values with missing FY
for y in range(len(Companies['Symbol'])):
    FY_Data = FY_Data.append({'Symbol':Companies['Symbol'][y],'Fiscal Year':'0000'}, ignore_index = True)
    for i in range(int(years.min()),2023):
        FY_Data = FY_Data.append({'Symbol':Companies['Symbol'][y],'Fiscal Year':str(i)}, ignore_index = True)

In [90]:
#Creates dataframe for revenue and profit data where fiscal year is 0000 (N/A)
fiscal_data = FY_Data.loc[FY_Data['Fiscal Year']=='0000'].merge(financials[['Symbol','Revenue(USD Millions)','Profit']], on='Symbol', how='left')
#Drops the rows with a FY of 0000 so the fiscal_data can be joined
FY_Data = FY_Data.drop(index=(FY_Data[FY_Data['Fiscal Year']=='0000']).index)

In [93]:
#Adds the fiscal_data to the FY_data dataframe
fiscal_data = fiscal_data.drop(['Profit_x','Revenue'], axis=1)
FY_Data = FY_Data.drop(['Profit','Revenue'], axis=1)
FY_Data = pd.concat([FY_Data, fiscal_data]).sort_values(by=['Symbol'])
FY_Data = FY_Data.rename(columns={'Revenue(USD Millions)':'Revenue','Profit_y':'Profit'})

In [97]:
#At this point, revenue and profit has been added to the table. This code joins the rest of the financial data based on year and symbol 
for i in range(len(financials_type)):
    df = pd.DataFrame(columns=['Symbol','Fiscal Year',financials_type[i]]) 
    df['Symbol'] = financials['Symbol']
    df['Fiscal Year'] = financials['{}_FY'.format(financials_type[i])] #Adds the FY for the specific financial type
    df[financials_type[i]] = financials['{}_Amount'.format(financials_type[i])] #Adds the amount for the specific financial type
    FY_Data = FY_Data.merge(df, on=['Symbol','Fiscal Year'], how='left')
FY_Data = FY_Data.drop(['Operating_Income_x','Net_Income_x','Assets_x','Equity_x'], axis=1)
types=[i+'_y' for i in financials_type]
types = types + ['Revenue','Profit']
FY_Data = FY_Data.dropna(axis=0,subset=types,how='all').reset_index() #drops data where there are no values for a fiscal year

In [100]:
#Transformed data to CSV
path=os.path.join(os.getcwd(),'FINANCIALS.csv')
path
csv = FY_Data.to_csv(path)

<h3>Data Upload</h3>
Uploads the data to the IBM DB2 database

In [74]:
#Loads data into the COMPANIES table
comp_columns = 'STOCK_SYMBOL,COMPANY_NAME,INDUSTRY_1,INDUSTRY_2,CITY,STATE,ACTIVE_IND'
for i,row in Companies.iterrows():
    symbol=row[1].strip()
    name=row[2].strip().replace("'",'')
    ind_1=row[3].strip().title()
    ind_2=row[4].strip().title()
    city=row[5].strip()
    state=row[6]
    #All companies are intitially set to inactive
    data = "'{}','{}','{}','{}','{}','{}','N'".format(symbol,name,ind_1,ind_2,city,state)
    sql = "INSERT INTO COMPANIES (" +comp_columns + ") VALUES("+data+")"
    ibm_db.exec_immediate(conn, sql)

In [70]:
#Loads data into the FINANCIALS table
fin_columns = 'STOCK_SYMBOL,FISCAL_YEAR,REVENUE,PROFIT,OPERATING_INCOME,NET_INCOME,ASSETS,EQUITY'
for i,row in FY_Data.iterrows():
    symbol = row[1]
    fy =str(row[2])
    revenue=str(row[3]).replace('nan','0')
    profit=str(row[4]).replace('nan','0')
    oincome=str(row[5]).replace('nan','0')
    nincome=str(row[6]).replace('nan','0')
    assets=str(row[7]).replace('nan','0')
    equity=str(row[8]).replace('nan','0')
    data = "'{}',{},{},{},{},{},{},{}".format(symbol,fy,revenue,profit,oincome,nincome,assets,equity)
    sql = "INSERT INTO FINANCIALS (" +fin_columns + ") VALUES("+data+")"
    ibm_db.exec_immediate(conn, sql)

In [71]:
ibm_db.close(conn)

True